# Biobabel example usage

## Basic data input and inspection

In [ ]:
import biobabel as bb

In [ ]:
bio = bb.load('../tests/samples/example.hdf5') # read example file included in the package (containing simulated data)

In [ ]:
bio.print()  # display a human-readable overview of the dataset

In [ ]:
bio.plot() # show a summary plot of all signals

In [ ]:
bio.get_participants() # the format can hold data for multiple participants at once, here two participants a and b

In [ ]:
bio.find_channels() # returns the IDs of the channels in the dataset. 
# Each channel has an ID (shown below) and associated meta data, as well as the actual data stream itself (not shown)

In [ ]:
bio.find_channels({"participant":"a"}) # find all channels for a particular participant only

In [ ]:
bio.find_channels({"modality":"ecg"}) # find all channels with a particular modality

In [ ]:
hdr,dat = bio.get('a_ecg')
hdr # find the associated metadata for this channel

In [ ]:
hdr

In [ ]:
dat

In [ ]:
t = bio.get_time('a_ecg') # reproduce a time vector (handy for plotting)

In [ ]:
import matplotlib.pyplot as plt
f,ax = plt.subplots(1,1,figsize=(5,3))
plt.plot(t,dat)
plt.xlabel('t (s)')

## Basic manipulations

### Copying

In [ ]:
# A deep copy of the data can be obtained via:
dupl = bio.copy()
# Deep copying means that anything you do to the new object (dupl) will not affect the original object (bio)

### Selecting channels

In [ ]:
# Basic usage of select function:
dupl.select('b_ecg')

In [ ]:
dupl.print()

In [ ]:
# You can also select by any metadata, here to select only the ECG channels
bio.copy().select({'modality':'ecg'}).print()

### Dropping channels

In [ ]:
dupl = bio.copy()
dupl.drop('b_ecg')

In [ ]:
dupl.print()

### Renaming data channels

In [ ]:
dupl.rename('a_ecg','ecg')
dupl.rename('a_ppg','ppg')
dupl.plot(figsize=(8,4))

### Cropping

In [ ]:
dupl.crop(2.5,8) # crop the data to the time interval from 2.5 to 8 seconds.

In [ ]:
dupl.print()

In [ ]:
dupl.plot(figsize=(8,4)) # note that the markers (dashed lines) have also been shifted, as they should be

In [ ]:
# Note that the original object is unchanged:
bio.print()

## Dataset output

In [ ]:
dupl.save('dataset_copy.hdf5') # save to a new file

# Preprocessing using third-party software

Some boilerplate code for automatic preprocessing using `neurokit2`. 

In [ ]:
import neurokit2
import biobabel as bb
x = bb.load('dataset_copy.hdf5')
prep = {}
for hdr,signal in x.find({'modality':'ecg'}):
    prep[hdr['id']] = neurokit2.ecg_process(signal,sampling_rate=hdr['sampling_frequency'])
# That's all folks!

In [ ]:
prep # see the results (for each column)

In [ ]:
# Visual representation of the automatically detected R peaks
x.clear_markers()
_,peaks = prep[hdr['id']]; x.add_marker('rpeak',peaks['ECG_R_Peaks']/1000)  # convert the peak times back to seconds from milliseconds
x.plot('ecg',figsize=(8,4))